In [ ]:
import polars as pl
from pathlib import Path

# CSV 파일 경로
CSV_DIR = Path(
    "/Users/nathan.song/projects/tasks/laplace-tasks/s3-data-migrator/data/source/SMARTSTORE_REST/2025-12-29"
)

# 헤더가 없는 CSV 파일들을 읽기 (glob 패턴 사용)
df = pl.scan_csv(CSV_DIR / "*.csv", has_header=False, new_columns=["s3_path"])

df.head(10).collect()

In [ ]:
# 파일명(user_id)도 함께 가져오기
df_with_source = pl.scan_csv(
    CSV_DIR / "*.csv",
    has_header=False,
    new_columns=["s3_path"],
    include_file_paths="source_file",
).with_columns(pl.col("source_file").str.extract(r"(\d+)\.csv$", 1).alias("user_id"))

df_with_source.head(10).collect()

In [ ]:
# 전체 행 수와 유니크 user_id 수 확인
print(f"Total rows: {df_with_source.select(pl.len()).collect().item():,}")
print(
    f"Unique users: {df_with_source.select(pl.col('user_id').n_unique()).collect().item():,}"
)

In [ ]:
df_with_source.group_by("user_id").agg(pl.col("s3_path").count()).sort(
    "s3_path", descending=True
).head(10).collect()

In [ ]:
import polars as pl

df = pl.read_parquet("./data/data_20251230_083437.parquet")
df